In [ ]:
!nvidia-smi

In [ ]:
!pip install numpy==1.26.4 matplotlib==3.9.2 --force-reinstall

In [ ]:
!pip install -U ultralytics

In [ ]:
import json
import os
from tqdm import tqdm
import shutil

In [ ]:
source_path = "/kaggle/input/flir-thermal-images-dataset/FLIR_ADAS_1_3/train/thermal_annotations.json"
destination_path = "/kaggle/working/flir_yolo/train/labels"

source_path2 = "/kaggle/input/flir-thermal-images-dataset/FLIR_ADAS_1_3/val/thermal_annotations.json"
destination_path2 = "/kaggle/working/flir_yolo/val/labels"

img_width = 640
img_height = 512

def convert(source_path, destination_path, img_width=640, img_height=512):
    os.makedirs(destination_path, exist_ok=True)

    with open(source_path, 'r') as f:
        data = json.load(f)

    PERSON_FLIR_ID = 1
    YOLO_CLASS_ID = 0
    
    print(f"Loaded {len(data['annotations'])} annotations from {source_path}")
    print(f"Converting to SINGLE YOLO CLASS (Person={YOLO_CLASS_ID}) format...\n")

    image_map = {img["id"]: img["file_name"] for img in data["images"]}

    counts = 0
    images_processed = set()

    for ann in tqdm(data["annotations"], desc="Processing Annotations"):
        cat_id = ann["category_id"]
        
        if cat_id != PERSON_FLIR_ID:
            continue

        img_id = ann["image_id"]
        if img_id not in image_map:
            continue

        img_name = os.path.splitext(os.path.basename(image_map[img_id]))[0]
        label_file = os.path.join(destination_path, f"{img_name}.txt")

        x, y, w, h = ann["bbox"]
        
        if w <= 0 or h <= 0:
             continue
        
        if x < 0 or y < 0 or x + w > img_width or y + h > img_height:
             continue
        
        x_center = (x + w / 2) / img_width
        y_center = (y + h / 2) / img_height
        w /= img_width
        h /= img_height
        
        if not (0.0 <= x_center <= 1.0 and 0.0 <= y_center <= 1.0 and 0.0 <= w <= 1.0 and 0.0 <= h <= 1.0):
             continue 

        with open(label_file, "a") as f:
            f.write(f"{YOLO_CLASS_ID} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

        counts += 1
        images_processed.add(img_name)

    print(f"\nConversion complete! {counts} annotations written for {len(images_processed)} images in {destination_path}")


convert(source_path, destination_path, img_width, img_height)
convert(source_path2, destination_path2, img_width, img_height)

In [ ]:
yaml_path = "/kaggle/working/flir_yolo/data.yaml"

os.makedirs("/kaggle/working/flir_yolo", exist_ok=True)

with open(yaml_path, "w") as f:
    f.write("""path: /kaggle/working/flir_yolo

train: train/images
val: val/images

names:
  0: person
""")

print(f"data.yaml created at: {yaml_path}")

In [ ]:
train_img_src = "/kaggle/input/flir-thermal-images-dataset/FLIR_ADAS_1_3/train/thermal_8_bit"
val_img_src = "/kaggle/input/flir-thermal-images-dataset/FLIR_ADAS_1_3/val/thermal_8_bit"

train_img_dst = "/kaggle/working/flir_yolo/train/images"
val_img_dst = "/kaggle/working/flir_yolo/val/images"

os.makedirs(train_img_dst, exist_ok=True)
os.makedirs(val_img_dst, exist_ok=True)

for src, dst in [(train_img_src, train_img_dst), (val_img_src, val_img_dst)]:
    for fname in os.listdir(src):
        os.symlink(os.path.join(src, fname), os.path.join(dst, fname))


print("Image folders ready!")

In [ ]:
!yolo detect train data=/kaggle/working/flir_yolo/data.yaml model=yolo11s.pt epochs=50 imgsz=640 workers=2 patience=5